<a href="https://colab.research.google.com/github/dkuzzmin/DI-Bootcamp/blob/main/Week8/Day3/W8D3_XC_XP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer


# Exploratory Data Analysis (EDA)

In [2]:
df = pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df = df.iloc[:len(df) // 5]


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     10000 non-null  object
 1   sentiment  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [5]:
df[df['sentiment'].isna()]

,review,sentiment


In [6]:
def count_words(review):
    return len(str(review).split())

In [7]:
df['words count'] = df['review'].apply(count_words)

<ipython-input-7-0dafb2cdb1bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['words count'] = df['review'].apply(count_words)


In [8]:
df.head(5)

,review,sentiment,words count
0,One of the other reviewers has mentioned that ...,positive,307
1,A wonderful little production. <br /><br />The...,positive,162
2,I thought this was a wonderful way to spend ti...,positive,166
3,Basically there's a family where a little boy ...,negative,138
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230


# Preprocessing

In [9]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')

def simple_preprocessing(text):
    nltk.download('stopwords')
    nltk.download('punkt')
    stopwords = nltk.corpus.stopwords.words('english')
    text = text.lower()
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'#|\@', '', text)
    text = re.sub(r'[^\w\s]', '', text)


    words = nltk.word_tokenize(text)
    custom_stopwords = set(stopwords)
    pretext = [word for word in words if word.lower() not in custom_stopwords]

    return ' '.join(pretext)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
for i, row in df.iterrows():
    text = row['review']
    df.at[i, 'prereview'] = simple_preprocessing(text)

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading packa

In [11]:
df.head()

,review,sentiment,words count,prereview
0,One of the other reviewers has mentioned that ...,positive,307,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,positive,162,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,166,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,138,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230,petter matteis love time money visually stunni...


In [12]:
len1 = len(df)
df = df.drop_duplicates('prereview')
print('Deleted duplicatd reviews', len1-len(df))

Deleted duplicatd reviews 17


In [13]:
def stemming(text):
    stemmer = PorterStemmer()
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [14]:
for i, row in df.iterrows():
    text = row['prereview']
    df.at[i, 'stems'] = stemming(text)

In [15]:
df

,review,sentiment,words count,prereview,stems
0,One of the other reviewers has mentioned that ...,positive,307,one reviewers mentioned watching 1 oz episode ...,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,162,wonderful little production filming technique ...,wonder littl product film techniqu unassum old...
2,I thought this was a wonderful way to spend ti...,positive,166,thought wonderful way spend time hot summer we...,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,138,basically theres family little boy jake thinks...,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230,petter matteis love time money visually stunni...,petter mattei love time money visual stun film...
...,...,...,...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive,148,fun entertaining movie wwii german spy julie a...,fun entertain movi wwii german spi juli andrew...
9996,Give me a break. How can anyone say that this ...,negative,218,give break anyone say good hockey movie know m...,give break anyon say good hockey movi know mov...
9997,This movie is a bad movie. But after watching ...,negative,228,movie bad movie watching endless series bad ho...,movi bad movi watch endless seri bad horror mo...
9998,This is a movie that was probably made to ente...,negative,136,movie probably made entertain middle school ea...,movi probabl made entertain middl school earli...


# Preparing Data To Train The Model

In [16]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})

In [17]:
df['sentiment'].unique()

array([1, 0])

In [18]:
X = df['stems']
Y = df['sentiment']

tfidf_vectorizer = TfidfVectorizer()
X_vectorized = tfidf_vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y, test_size=0.3, random_state=42)

In [19]:
print(X_train.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_test))

(6988, 49882)
(2995, 49882)
6988
2995


# Machine Learning Model: Instantiating, Training, Predicting And Evalueting

In [20]:
logistic_regression_model = LogisticRegression(random_state=42)

logistic_regression_model.fit(X_train, y_train)

y_pred = logistic_regression_model.predict(X_test)

In [21]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

print()
print('Classification Report:')
print(classification_report(y_test, y_pred))
print()

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8747913188647746

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.85      0.87      1488
           1       0.86      0.90      0.88      1507

    accuracy                           0.87      2995
   macro avg       0.88      0.87      0.87      2995
weighted avg       0.88      0.87      0.87      2995


Confusion Matrix:
[[1266  222]
 [ 153 1354]]


# Predicting A New Review

In [28]:
reviews = ["I loved this movie!", "This movie was a bad comedy movie!"]

preprocessed_reviews = [simple_preprocessing(x) for x in reviews]
new_reviews_vectorized = tfidf_vectorizer.transform(preprocessed_reviews)
res=pd.DataFrame(reviews, columns=["review"])
res['predictions'] = logistic_regression_model.predict(new_reviews_vectorized)
res['sentiments']= ['negative' if pred == 0 else 'positive' for pred in res['predictions']]

res

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,review,predictions,sentiments
0,I loved this movie!,1,positive
1,This movie was a bad comedy movie!,0,negative
